# Project Part 2

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/entrylevelcs/CS39AA-Project/blob/main/project_part2.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/entrylevelcs/CS39AA-Project/blob/main/project_part2.ipynb)


## 1. Introduction/Background

For this part of the project we are taking the nearest neighbors model and training it using both our AI and real data sets. From this simple model we can see how well each of the different training data sets do after being optimizing the hyperparameters of this simple model.

## 2. Simple Modeling with KNeighborsClassifier

In [1]:
# import all of the python modules/packages you'll need here
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# ...

Get the data set from real steam reviews. The specific set of data that I am using for this notebook comes from https://www.kaggle.com/datasets/andrewmvd/steam-reviews/ but is just a sample of 25000 from the entire set.

In [2]:
human_data = 'https://raw.githubusercontent.com/entrylevelcs/CS39AA-Project/main/human_dataset.csv'
df = pd.read_csv(human_data)
df = df[df["review_text"].notnull()]
sample_size = len(df)
#sample_size = 25000
#df = df.sample(sample_size)
# previous two lines are for sampling from the entire human dataset

In [3]:
df.head()

,Unnamed: 0,app_id,app_name,review_text,review_score,review_votes
0,949219,212680,FTL: Faster Than Light,Really gg.,1,0
1,1151136,218620,PAYDAY 2,"I'm not entirely sure what I was expecting, bu...",-1,0
2,4031479,31280,Poker Night at the Inventory,"First I thought I was gonna buy this, play so ...",1,0
3,6382952,94400,Nidhogg,Stab your opponent repeatedly and win the glor...,1,0
4,3770644,300060,ADR1FT,I think this experience is pretty much defined...,1,0


Get the data set that was generated by chatgpt. This review data started as only being about CS:GO but has been expanded to be more general and talk about other games.

In [4]:
ai_data = 'https://raw.githubusercontent.com/entrylevelcs/CS39AA-Project/main/gpt3.5_generated_data.csv'
df2 = pd.read_csv(ai_data)

Sets up our different inputs and outputs. X and y are the inputs and outputs from the real steam reviews while X1 and y1 are from the AI generated reviews. I also adjusted the test size so that the training data size the same length as the AI generated data set. From the shape of each we can see that the vocab size is 11127 words.

In [5]:
X = df['review_text'].copy()
y = df['review_score'].copy()
X1 = df2['Review'].copy()
y1 = df2[' Sentiment'].copy()
ai_data_size = X1.size

X_train_raw, X_val_raw, y_train, y_val = train_test_split(X, y, test_size=((sample_size-ai_data_size)/sample_size), random_state=21)

In [7]:
real_data = X_train_raw.copy()
real_data['scores'] = y_train.copy()

In [8]:
import torch
from transformers import AutoTokenizer, GPT2ForTokenClassification,  TrainingArguments, Trainer
from datasets import Dataset, load_metric

tokenizer = AutoTokenizer.from_pretrained("gpt2")
realModel = GPT2ForTokenClassification.from_pretrained("gpt2")

c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
df_raw = Dataset.from_pandas(real_data)

def tokenize_function(examples):
    return tokenizer(examples['review_text'], truncation=True, max_length=50)

df = df_raw.map(tokenize_function, batched=True)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Muajeh Lee\AppData\Local\Temp\ipykernel_2196\2853803468.py", line 1, in <module>
    df_raw = Dataset.from_pandas(real_data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\datasets\arrow_dataset.py", line 846, in from_pandas
    table = InMemoryTable.from_pandas(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\datasets\table.py", line 747, in from_pandas
    return cls(pa.Table.from_pandas(*args, **kwargs))
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pyarrow\table.pxi", line 3557, in pyarrow.lib.Table.from_pandas
  File "c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py", line 570, in dataframe_to_arrays
    convert_fields) = _get_c

In [11]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=10,
                                  do_train=True,
                                  report_to=None,
                                  output_dir="realData",
                                  evaluation_strategy="steps",
                                  eval_steps=200,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

trainer = Trainer(model = realModel, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)

NameError: name 'ds_train' is not defined